In [ ]:
# !python3 -m pip install --user neo elephant viziphant

#  Spike PAtterns DEtection using SPADE


In [ ]:
import numpy as np
import quantities as pq
import neo
import elephant
import viziphant
np.random.seed(4542)

## using the tutorial

There are different tutorials such as this one on [ASSET](https://elephant.readthedocs.io/en/latest/tutorials/asset.html) - or this more directly related about [3D SPADE](https://elephant.readthedocs.io/en/latest/tutorials/spade.html) from which we take these first cells..

We start by buiding $10$ spike trains with some synchrony


In [ ]:
from elephant.spike_train_generation import compound_poisson_process
compound_poisson_process?

 From the tutorial:
 
 > to a distribution A[n] indicating the likelihood of finding synchronous spikes of a given order n. In our example, we construct the distribution such that we have a small probability to produce a synchronous event of order 10 (A[10]==0.02). Otherwise spikes are not synchronous with those of other neurons (i.e., synchronous events of order 1, A[1]==0.98). Notice that the length of the distribution A determines the number len(A)-1 of spiketrains returned by the function, and that A[0] is ignored for reasons of clearer notation.

In [ ]:
A = [0]+[0.98]+[0]*8+[0.02]

In [ ]:
spiketrains = elephant.spike_train_generation.compound_poisson_process(
   rate=5*pq.Hz, A=A, t_stop=10*pq.s)
len(spiketrains)

https://viziphant.readthedocs.io/en/latest/


In [ ]:
from viziphant.rasterplot import rasterplot
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,3))
rasterplot(spiketrains, axes=ax)

In [ ]:
A

In [ ]:
# rasterplot?

In [ ]:
for i in range(90):
    spiketrains.append(elephant.spike_train_generation.homogeneous_poisson_process(
        rate=5*pq.Hz, t_stop=10*pq.s))

In [ ]:
st = spiketrains[0]
st.times

In [ ]:
patterns = elephant.spade.spade(
    spiketrains=spiketrains, binsize=1*pq.ms, winlen=1, min_spikes=3,
    n_surr=100, dither=5*pq.ms,
    psr_param=[0,0,0],
    output_format='patterns')['patterns']

In [ ]:
patterns

In [ ]:
viziphant.patterns.plot_patterns(spiketrains, patterns)

## scanning some parameters

the above as one function:

In [ ]:
A_sync = 0.02
shift = 1.0

def generate_and_test(A_sync=A_sync, shift=shift):
    A = [0]+[1.-A_sync]+[0]*8+[A_sync]
    spiketrains = elephant.spike_train_generation.compound_poisson_process(
                             rate=5*pq.Hz, A=A, shift=shift*pq.ms, t_stop=10*pq.s)
    for i in range(90):
        spiketrains.append(elephant.spike_train_generation.homogeneous_poisson_process(
            rate=5*pq.Hz, t_stop=10*pq.s))
    patterns = elephant.spade.spade(
                                spiketrains=spiketrains, binsize=1*pq.ms, winlen=1, min_spikes=3,
                                n_surr=100,dither=5*pq.ms,
                                psr_param=[0,0,0],
                                output_format='patterns')['patterns']
    return patterns


* changing the synchrony probability to check when SPADE will begin to fail:


In [ ]:
for A_sync_ in np.logspace(-2, -.5, 10):
    patterns = generate_and_test(A_sync=A_sync_)
    print(patterns)


* adding a random shift to check robustness of SPADE:


In [ ]:
for shift_ in np.logspace(-1, 1, 10, base=10, endpoint=True):
    patterns = generate_and_test(shift=shift_)
    print(patterns)